In [1]:
"""
Collaborative Filtering ALS Recommender System using Spark MLlib adapted from
the Spark Summit 2014 Recommender System training example.

Developed By: Pranav Masariya
Supervisor: Dr. Magdalini Eirinaki
"""

import os
import numpy as np
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from pyspark.mllib.recommendation import ALS
from pyspark.ml.recommendation import ALS as mlals
from pyspark.ml.evaluation import RegressionEvaluator

import math
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
# Calling spark session to register application
spark = SparkSession \
    .builder \
    .appName("Recom") \
    .config("spark.recom.demo", "1") \
    .getOrCreate()

In [9]:
"""
Loading and Parsing Dataset
    Each line in the ratings dataset (ratings.csv) is formatted as:
         userId,movieId,rating,timestamp
    Each line in the movies (movies.csv) dataset is formatted as:
        movieId,title,genres

""" 

# Load ratings
ratings_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("movies.csv")

In [10]:
ratings_df

DataFrame[userId: int, movieId: int, rating: int]

In [11]:
"""
For the simplicity of this tutorial
    For each line in the ratings dataset, we create a tuple of (UserID, MovieID, Rating). 
    We drop the timestamp because we do not need it for this recommender.
"""

#ratings_df = ratings_df.drop('timestamp')
ratings_df.show(30)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|   196|    242|     3|
|   186|    302|     3|
|    22|    377|     1|
|   244|     51|     2|
|   166|    346|     1|
|   298|    474|     4|
|   115|    265|     2|
|   253|    465|     5|
|   305|    451|     3|
|     6|     86|     3|
|    62|    257|     2|
|   286|   1014|     5|
|   200|    222|     5|
|   210|     40|     3|
|   224|     29|     3|
|   303|    785|     3|
|   122|    387|     5|
|   194|    274|     2|
|   291|   1042|     4|
|   234|   1184|     2|
|   119|    392|     4|
|   167|    486|     4|
|   299|    144|     4|
|   291|    118|     2|
|   308|      1|     4|
|    95|    546|     2|
|    38|     95|     5|
|   102|    768|     2|
|    63|    277|     4|
|   160|    234|     5|
+------+-------+------+
only showing top 30 rows



In [ ]:
# Load movies
movies_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("ml-latest-small/movies.csv")

In [ ]:
movies_df.show(5)

In [ ]:
"""
For each line in the movies dataset, we create a tuple of (MovieID, Title). 
    We drop the genres because we do not use them for this recommender.
"""
movies_df = movies_df.drop('genres')
movies_df.show(5)

In [12]:
"""
In order to determine the best ALS parameters, we will use the small dataset. 
We need first to split it into train, validation, and test datasets.
"""
(trainingData,validationData,testData) = ratings_df.randomSplit([0.6,0.2,0.2])

In [13]:
# Prepare test and validation set. They should not have ratings

validation_for_predict = validationData.select('userId','movieId')
test_for_predict = testData.select('userId','movieId')

In [15]:
"""
Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by 
using Alternating Least Squares. The implementation in MLlib has the following parameters:

    1. numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
    2. rank is the number of latent factors in the model.
    3. iterations is the number of iterations to run.
    4. lambda specifies the regularization parameter in ALS.
    5. implicitPrefs specifies whether to use the explicit 
        feedback ALS variant or one adapted for implicit feedback data.
    6. alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline 
        confidence in preference observations.

"""

seed = 5 #Random seed for initial matrix factorization model. A value of None will use system time as the seed.
iterations = 10
regularization_parameter = 0.1 #run for different lambdas - e.g. 0.01
ranks = [4, 8, 12] #number of features
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1

In [20]:
# Let us traing our dataset and check the best rank with lowest RMSE
# predictAll method of the ALS takes only RDD format and hence we need to convert our dataframe into RDD
# df.rdd will automatically converts Dataframe into RDD

for rank in ranks:
    model = ALS.train(trainingData, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict.rdd).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validationData.rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()) # RMSE Error
    errors[err] = error
    err += 1
    print ('For rank %s the RMSE is %s' % (rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank

print ('The best model was trained with rank %s' % best_rank)

For rank 4 the RMSE is 0.9477245381450695
For rank 8 the RMSE is 0.9532198469578983
For rank 12 the RMSE is 0.9526420573918778
The best model was trained with rank 4


In [ ]:
"""
Spark will soon deprecate MLLIb package. 
They are focusing more on ML packages with standard machine learning implementation
Let's see that package also
"""
als =  mlals(maxIter=iterations,rank=4,seed=seed,regParam=regularization_parameter, userCol="userId", itemCol="movieId",ratingCol="rating")
modelML = als.fit(trainingData)
pred = modelML.transform(validationData)
pred = pred.where(pred['prediction'] != 'NaN')
    
# Evaluate the model by computing RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(pred)

print 'RMSE is %s' % rmse

"""
The best part is we do not have to worry about RDD any more with this library
"""

In [21]:
# Let's take test dataset and get ratings
predictions_test = model.predictAll(test_for_predict.rdd).map(lambda r: ((r[0], r[1]), r[2]))

In [22]:
## visualize preditions, here third element is predictions generated by ALS Model
predictions_test.take(3)

[((292, 320), 4.163789684707593),
 ((13, 320), 2.4530965672566305),
 ((118, 320), 4.9492797323585265)]

In [23]:
"""
Let's start recommending movies.
I have written a method to call recommendations for a perticular user from test data

TODO: You need to execute one more step before calling getRecommendations, 
      Think about that step. If you go through the seps below, you will realize it soon.
"""
def getRecommendations(user,testDf,trainDf,model):
    # get all user and his/her rated movies
    userDf = testDf.filter(testDf.userId == user)
    # filter movies from main set which have not been rated by selected user
    # and pass it to model we sreated above
    mov = trainDf.select('movieId').subtract(userDf.select('movieId'))
    
    # Again we need to covert our dataframe into RDD
    pred_rat = model.predictAll(mov.rdd.map(lambda x: (user, x[0]))).collect()
    
    # Get the top recommendations
    recommendations = sorted(pred_rat, key=lambda x: x[2], reverse=True)[:50]
    
    return recommendations

In [24]:
# Assign user id for which we need recommendations
user = 336

# Call getRecommendations method
derived_rec = getRecommendations(user,testData,trainingData,model)

print ("Movies recommended for:%d" % user)



Movies recommended for:336


In [27]:
derived_rec


[Rating(user=336, product=1368, rating=5.1981050690456625),
 Rating(user=336, product=958, rating=4.814280433278696),
 Rating(user=336, product=272, rating=4.709120785820507),
 Rating(user=336, product=1282, rating=4.687403767642065),
 Rating(user=336, product=315, rating=4.654853148942708),
 Rating(user=336, product=64, rating=4.611159440477497),
 Rating(user=336, product=192, rating=4.57882490756566),
 Rating(user=336, product=316, rating=4.555875752573431),
 Rating(user=336, product=357, rating=4.534705183664028),
 Rating(user=336, product=1643, rating=4.524863260473367),
 Rating(user=336, product=127, rating=4.521642274576629),
 Rating(user=336, product=1639, rating=4.497326809163149),
 Rating(user=336, product=954, rating=4.4886930781352214),
 Rating(user=336, product=1512, rating=4.4623095886606015),
 Rating(user=336, product=318, rating=4.456360167609304),
 Rating(user=336, product=114, rating=4.454167268584019),
 Rating(user=336, product=302, rating=4.430053560990084),
 Rating(

In [ ]:
# Print the result
# TODO: we can convert derived_rec into a dataframe to present it properly
for i in xrange(len(derived_rec)):
    print i+1
    movies_df.filter(movies_df.movieId==derived_rec[i][1]).select('title').show()